##### Copyright 2025 Google LLC.

In [ ]:
# @title Apache License 2.0 라이선스
# 이 코드는 Apache License 2.0에 따라 사용이 허가됩니다.
# 라이선스 전문: https://www.apache.org/licenses/LICENSE-2.0

# 🎙️ Gemini API: 텍스트 음성 변환 (TTS) 시작하기

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started_TTS.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

Gemini API는 텍스트 입력을 **단일 화자** 또는 **다중 화자 오디오**(NotebookLM의 팟캐스트 기능과 유사)로 변환할 수 있습니다.

이 노트북에서는 Gemini 모델의 **TTS(Text-to-Speech)** 기능을 제어하고, 스타일, 억양, 속도, 톤을 조절하는 방법을 배웁니다.

코드를 실행하기 전에 [AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemini-2.5-flash-preview-tts)에서 먼저 체험해보세요!

---

⚠️ **중요**: TTS 모델은 오직 텍스트를 음성으로 변환하는 기능만 합니다. 일반 Gemini 모델처럼 추론 기능이 없습니다.
- ✅ 가능: "이 문장을 이런 스타일로 말해줘"
- ❌ 불가능: "하늘이 왜 파란색인지 설명해줘"

추론 기능이 필요하다면 [Live API](./Get_started_LiveAPI.ipynb)를 사용하세요.

📖 [공식 문서](https://ai.google.dev/gemini-api/docs/audio-generation)에서 더 자세한 정보를 확인할 수 있습니다.

<table align="left" border="3">
  <tr>
    <td bgcolor="#DCE2FF">
      <font size=30>🪧</font>
    </td>
    <td bgcolor="#DCE2FF">
      <h4><font color=black>오디오 출력은 프리뷰 기능입니다. 현재는 쿼터 제한 내에서 무료로 사용 가능하지만, 향후 변경될 수 있습니다.</font></h4>
    </td>
  </tr>
</table>

---
## 🛠️ 설정

### API 키 설정

아래 셀을 실행하려면 API 키가 `GOOGLE_API_KEY`라는 이름의 Colab Secret에 저장되어 있어야 합니다.

API 키가 없거나 Colab Secret 생성 방법을 모르시면 [인증 가이드](../quickstarts/Authentication.ipynb)를 참조하세요.

In [ ]:
from google.colab import userdata

# Colab Secret에서 API 키 가져오기
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

### SDK 설치 및 초기화

In [ ]:
# 다중 화자 오디오를 위해 1.16 이상 버전 필요
!pip install -U -q "google-genai>=1.16.0"

In [ ]:
from google import genai
from google.genai import types

# Gemini 클라이언트 초기화
client = genai.Client(api_key=GOOGLE_API_KEY)

### 모델 선택

오디오 출력은 `tts` 모델만 지원합니다:
- `gemini-2.5-flash-preview-tts`
- `gemini-2.5-pro-preview-tts`

모든 Gemini 모델에 대한 자세한 정보는 [공식 문서](https://ai.google.dev/gemini-api/docs/models/gemini)를 참조하세요.

In [ ]:
# 사용할 TTS 모델 선택
MODEL_ID = "gemini-2.5-flash-preview-tts"  # @param ["gemini-2.5-flash-preview-tts","gemini-2.5-pro-preview-tts"] {"allow-input":true, isTemplate: true}

### 도우미 함수 정의

모델에 프롬프트를 전달하고 노트북에서 오디오를 재생하기 위한 헬퍼 함수를 만듭니다.

In [ ]:
# @title 도우미 함수 (이 셀을 실행하세요)

import contextlib
import wave
from IPython.display import Audio

file_index = 0

@contextlib.contextmanager
def wave_file(filename, channels=1, rate=24000, sample_width=2):
    """WAV 파일을 생성하는 컨텍스트 매니저"""
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)    # 채널 수 (모노=1)
        wf.setsampwidth(sample_width) # 샘플 너비 (바이트)
        wf.setframerate(rate)         # 샘플 레이트 (Hz)
        yield wf

def play_audio_blob(blob):
    """오디오 데이터를 파일로 저장하고 재생"""
    global file_index
    file_index += 1

    fname = f'audio_{file_index}.wav'
    with wave_file(fname) as wav:
        wav.writeframes(blob.data)

    return Audio(fname, autoplay=True)

def play_audio(response):
    """API 응답에서 오디오를 추출하여 재생"""
    return play_audio_blob(response.candidates[0].content.parts[0].inline_data)

---
## 🎤 기본 음성 생성

간단한 텍스트를 음성으로 변환해봅시다!

In [ ]:
# 기본 TTS 요청
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Say '안녕하세요, 저는 Gemini입니다!'",  # 한국어도 가능!
    config={"response_modalities": ['Audio']},  # 응답 형식을 오디오로 설정
)

생성된 출력은 응답의 `inline_data`에 있으며, 이것이 실제 오디오 데이터입니다.

In [ ]:
# 응답에서 오디오 데이터 확인
blob = response.candidates[0].content.parts[0].inline_data
print(f"MIME 타입: {blob.mime_type}")

In [ ]:
# 생성된 오디오 재생
play_audio_blob(blob)

💡 **팁**: TTS 모델은 텍스트 읽기만 가능하므로, 항상 "say", "read", "TTS" 같은 명령어를 사용해야 합니다. 그렇지 않으면 아무것도 생성되지 않습니다.

---
## 🎚️ 음성 제어하기

30가지 내장 음성과 24개 지원 언어를 조합하여 다양한 음성을 만들 수 있습니다.

### 음성(Voice) 선택하기

30가지 음성 중에서 선택하세요. 각 음성의 특성은 [공식 문서](https://ai.google.dev/gemini-api/docs/speech-generation#voices)에서 확인할 수 있습니다.

In [ ]:
# 사용할 음성 선택 (30가지 중 하나 선택)
voice_name = "Sadaltager"  # @param ["Zephyr", "Puck", "Charon", "Kore", "Fenrir", "Leda", "Orus", "Aoede", "Callirhoe", "Autonoe", "Enceladus", "Iapetus", "Umbriel", "Algieba", "Despina", "Erinome", "Algenib", "Rasalgethi", "Laomedeia", "Achernar", "Alnilam", "Schedar", "Gacrux", "Pulcherrima", "Achird", "Zubenelgenubi", "Vindemiatrix", "Sadachbia", "Sadaltager", "Sulafar"]

response = client.models.generate_content(
    model=MODEL_ID,
    contents="""Say '저는 매우 박식한 모델이에요, 특히 그라운딩을 사용할 때요', 5초 기다린 후 '그렇게 생각하지 않으세요?'라고 말해줘.""",
    config={
        "response_modalities": ['Audio'],
        "speech_config": {
            "voice_config": {
                "prebuilt_voice_config": {
                    "voice_name": voice_name  # 선택한 음성 적용
                }
            }
        }
    },
)

play_audio(response)

### 언어 변경하기

모델에게 특정 언어로 말하라고 지시하면 됩니다! [문서](https://ai.google.dev/gemini-api/docs/speech-generation#languages)에서 지원되는 언어 목록을 확인하세요.

**한국어도 지원됩니다! 🇰🇷**

In [ ]:
# 한국어로 읽기
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    다음을 한국어로 읽어주세요:

    안녕하세요! AI City Builders에 오신 것을 환영합니다.
    오늘도 AI와 함께 멋진 하루 보내세요!
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

In [ ]:
# 프랑스어로 읽기 (원본 예제)
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    Read this in French:

    Les chaussettes de l'archiduchesse sont-elles sèches ? Archi-sèches ?
    Un chasseur sachant chasser doit savoir chasser sans son chien.
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

### 말하는 방식 제어하기 (스타일, 톤, 억양, 속도)

자연어 프롬프트를 사용하여 스타일, 톤, 억양, 속도를 제어할 수 있습니다!

In [ ]:
# 🎃 으스스한 속삭임 스타일
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    Say in a spooky whisper (으스스한 속삭임으로):
    "어둠 속에서 무언가가 다가온다...
    뒤를 돌아보지 마세요..."
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

In [ ]:
# ⚡ 빠른 속도로 읽기 (면책조항 스타일)
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    Read this disclaimer in as fast a voice as possible while remaining intelligible:
    (이 면책조항을 알아들을 수 있는 한 최대한 빠르게 읽어주세요)

    본 사이트의 내용에 대한 오류나 누락에 대해 어떠한 책임도 지지 않습니다.
    이 사이트에 포함된 정보는 완전성, 정확성, 유용성 또는 적시성에 대한 보장 없이 '있는 그대로' 제공됩니다.
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

---
## 👥 다중 화자 (Multi-Speaker)

TTS 모델은 2명의 화자가 대화하는 콘텐츠도 생성할 수 있습니다!
(NotebookLM의 팟캐스트 기능처럼요)

화자가 두 명이라고 지정하면 됩니다:

In [ ]:
# 두 화자의 대화 생성
response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    Speaker1은 피곤하고 지루한 목소리로, Speaker2는 신나고 행복한 목소리로:
    (Make Speaker1 sound tired and bored, and Speaker2 sound excited and happy)

    Speaker1: 그래서... 오늘 일정이 뭐야?
    Speaker2: 절대 못 맞힐걸!
    """,
    config={"response_modalities": ['Audio']},
)

play_audio(response)

### 화자별 음성 지정하기

각 참여자의 음성을 선택하고 이름을 지정할 수 있습니다.

먼저 두 AI 전문가의 대화 스크립트를 생성해봅시다:

In [ ]:
# Gemini로 대화 스크립트 생성
transcript = client.models.generate_content(
    model='gemini-2.5-flash',
    contents="""
      안녕하세요! AI 전문가인 제이 박사와 그의 조수 민지의 팟캐스트에서
      발췌한 것처럼 들리는 짧은 대화(약 100단어)를 생성해주세요.
      주제는 'AI로 수익 창출하기'입니다.
      한국어로 작성해주세요.
    """
).text

print("📝 생성된 대화 스크립트:")
print(transcript)

이제 원하는 음성을 사용하여 TTS 모델로 대화를 렌더링합니다:

In [ ]:
# 다중 화자 설정 구성
config = types.GenerateContentConfig(
    response_modalities=["AUDIO"],
    speech_config=types.SpeechConfig(
        multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
            speaker_voice_configs=[
                # 제이 박사 음성 설정
                types.SpeakerVoiceConfig(
                    speaker='제이 박사',  # 화자 이름
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Orus',  # 차분하고 전문적인 음성
                        )
                    )
                ),
                # 민지 음성 설정
                types.SpeakerVoiceConfig(
                    speaker='민지',  # 화자 이름
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Leda',  # 밝고 활기찬 음성
                        )
                    )
                ),
            ]
        )
    )
)

# 대화 TTS 생성
response = client.models.generate_content(
    model=MODEL_ID,
    contents="TTS the following conversation between 제이 박사 and his assistant 민지: " + transcript,
    config=config,
)

play_audio(response)

---
## 🎯 다음 단계

이제 다중 화자 대화를 생성하는 방법을 배웠습니다! 다음 멋진 기능들도 시도해보세요:

| 기능 | 설명 | 노트북 |
|------|------|--------|
| 🎵 **음악 생성** | 음성 대신 음악 생성하기 | [Lyria RealTime](./Get_started_LyriaRealTime.ipynb) |
| 🖼️ **이미지 생성** | AI로 이미지 만들기 | [Imagen](./Get_started_imagen.ipynb) |
| 🎬 **비디오 생성** | AI로 비디오 만들기 | [Veo](./Get_started_Veo.ipynb) |
| 🎧 **오디오 이해** | Gemini로 오디오 파일 분석하기 | [Audio](./Audio.ipynb) |
| 💬 **실시간 대화** | Gemini와 실시간으로 대화하기 | [Live API](./Get_started_LiveAPI.ipynb) |

---

### 📚 유용한 링크

- [Gemini API 문서](https://ai.google.dev/gemini-api/docs)
- [TTS 음성 목록](https://ai.google.dev/gemini-api/docs/speech-generation#voices)
- [지원 언어 목록](https://ai.google.dev/gemini-api/docs/speech-generation#languages)
- [AI Studio](https://aistudio.google.com/)

---

🎉 **축하합니다!** Gemini TTS 시작 가이드를 완료했습니다!